# Plasma Coupling Using COMSOL Results

In this example, we use a COMSOL front-face coupling calculation provided by ORNL, exported as a standard Touchstone file.

The Touchstone file is first import as a [scikit-rf](https://scikit-rf.org/) [Network](https://scikit-rf.readthedocs.io/en/latest/tutorials/Networks.html), which is then modified to fit the WEST ICRH antenna electrical model requirements.

# Importing the S-parameters in the electric model

In [ ]:
import numpy as np
import skrf as rf

# WEST ICRH Antenna package
import sys; sys.path.append('..')
from west_ic_antenna import WestIcrhAntenna

In [ ]:
front_face_conventional = rf.Network(
    '../west_ic_antenna/data/Sparameters/front_faces/COMSOL/ORNL_front_face_conventional.s4p')
print(front_face_conventional)  # 50 Ohm S-param component at a single frequency of 55 MHz

The ports have been defined as:
<img src="COMSOL_WEST_port_index.png" />


So before to use the S-parameters directly to feed the electrical model, we need to:
- deembed the ports by 0.3m.
- renomalize port reference impedance to the front-face coax characteristic impedances. 
- reverse ports 2 and 3 to match the expected definition by the electrical model.

In [ ]:
# creating a 50 Ohm dummy coax line to be removed from the front face 
media_coax = rf.DefinedGammaZ0(frequency=front_face_conventional.frequency) # 50 Ohm TEM media
extra_line = media_coax.line(d=0.3, unit='m')
# deembed all the 4 ports
for port_idx in range(4):
    front_face_conventional = rf.connect(front_face_conventional, port_idx, extra_line.inv, 0)

The COMSOL S-parameters have been exported using a 50 Ohm reference impedance. However, we expect the port reference impedance equals to the characteristic impedance, that is, about 46.64 ohm, so we renormalize the Network to fit this need:

In [ ]:
front_face_conventional.renormalize(46.64)  # done inplace

And finally, for historical reasons (may change one day...), the S-matrix port ordering should be adjusted:

In [ ]:
front_face_conventional.renumber([1, 2], [2, 1]) # done inplace

OK, so now we can create the WEST antenna object:

In [ ]:
ant = WestIcrhAntenna(front_face=front_face_conventional,
                     frequency=front_face_conventional.frequency) # restrict to single frequ

Let's match the antenna for this coupling:

In [ ]:
Cs = ant.match_both_sides(f_match=55e6)

The coupling resistance of the antenna for this coupling in a nominal dipole excitation is:

In [ ]:
power = [1, 1]
phase = [0, np.pi]

# Coupling resistance
ant.Rc(power, phase)

The total voltages and currents at the capacitors are:

In [ ]:
power = [1.6/2, 1.6/2]  # MW, to adjust to fit with experiment
phase = [0, np.pi]  # rad

abs(ant.voltages(power, phase))  # results in kV

In [ ]:
abs(ant.currents(power, phase))  # results in kA

## Exporting voltage excitations
Now that the electrical model has been created and the antenna matched, one can export the voltage values at the front-face port into COMSOL to visualize the electric field and currents in the antenna front face and in the plasma.

Depending of the needs, the total voltages at the front-face port can be splitted into forward and reflected voltages:
$$
V = V_{fwd} + V_{ref} 
$$

In [ ]:
V_fwd, V_ref = ant.front_face_voltage_waves(power, phase, Cs=Cs)
print(V_fwd)

Of course, we find the same total voltage:

In [ ]:
Vtot = V_fwd + V_ref
V = ant.voltages(power, phase, Cs=Cs)
# pay attention that the voltage index differ from the front-face port indexes...
np.allclose(Vtot[:,[0,2,1,3]], V, rtol=1e-5)

It is also possible to deduce the forward powers and phases to setup on the four ports (assuming the reference impedance is real):

In [ ]:
powers, phases = ant.front_face_powers_phases(power, phase, Cs=Cs)
print(powers) # in Watt
print(phases) # in degrees